In [1]:
%load_ext autoreload
%autoreload 2

# Comparing model-generated ruggedness and sequence-level ruggedness

In [2]:
import os
import pandas as pd
import jax
import numpy as np
from synbio_morpher.utils.results.analytics.naming import get_true_interaction_cols


jax.config.update('jax_platform_name', 'cpu')

In [3]:
# Load dataset 1127
fn_1127 = os.path.join('..', 'data', 'raw', 'summarise_simulation', '2024_11_27_145142', 'tabulated_mutation_info.csv')
ds_1127 = pd.read_csv(fn_1127)

# Load dataset 0211
fn_0211 = os.path.join('data', 'simulate_circuits', '2025_02_11__13_56_51', 'tabulated_mutation_info.json')
ds_0211 = pd.read_json(fn_0211)

# Load model for ruggedness


# Load previously verified ruggedness fake circuits


In [26]:
# ds_1127.melt(id_vars='circuit_name', value_vars='mutation_num', var_name='interaction', value_name='interaction_value')
np.array(ds_1127[(ds_1127['sample_name'] == 'RNA_2') & (ds_1127['mutation_num'] > 0)].groupby('circuit_name').apply(lambda x: x['mutation_num'].values).to_list()).shape

/tmp/ipykernel_3824909/885465338.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  np.array(ds_1127[(ds_1127['sample_name'] == 'RNA_2') & (ds_1127['mutation_num'] > 0)].groupby('circuit_name').apply(lambda x: x['mutation_num'].values).to_list()).shape


(5000, 168)

In [28]:
# Calculate approximate ruggedness score for all circuits based on difference to mutated circuits
# Weight the metric by the number of sequence mutations, eg: dm = d1m / 1 + d2m / 2 + d5m / 5

cols_e = get_true_interaction_cols(
    ds_1127, 'energies', remove_symmetrical=True)
diffs_e = ds_1127[ds_1127['sample_name'] == 'RNA_2'].groupby('circuit_name').apply(
    lambda x: x[x['mutation_name'] == 'ref_circuit'][cols_e].values
    - x[x['mutation_name'] != 'ref_circuit'][cols_e].values)
diffs_e = np.array(diffs_e.to_list())
mut_nums = np.array(ds_1127[(ds_1127['sample_name'] == 'RNA_2') & (ds_1127['mutation_num'] > 0)].groupby(
    'circuit_name').apply(lambda x: x['mutation_num'].values).to_list())
rugg_seq = np.sum(diffs_e / mut_nums[..., None], axis=-1)

# Plot seq-based rugg and param-based rugg on x y plot -> should be linear

# Plot intensity of energy changed vs. sequence change

# Run full model on ds 0211 + 1127 and compare latent spaces
# Do clusters of motifs

/tmp/ipykernel_3824909/1205016581.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  diffs_e = ds_1127[ds_1127['sample_name'] == 'RNA_2'].groupby('circuit_name').apply(
/tmp/ipykernel_3824909/1205016581.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  'circuit_name').apply(lambda x: x['mutation_num'].values).to_list())
